In [58]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
import scipy.optimize as opt
from scipy.optimize import differential_evolution
import concurrent.futures
from tradingview_screener import Query
import rookiepy


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")


In [59]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [60]:
    
def screen_stocks():
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))

    # Initialize the query
    query = Query()

    # Select the fields you want to retrieve
    query = query.select('name', 'close', 'change', 'change_1m')

    # Apply the filters
    query = query.where('close', '<', 75) 
                # .where('change_1m', '>', 0) \
                # .where('change', '<', 2)

    # Retrieve the data with cookies
    n_rows, df = query.get_scanner_data(cookies=cookies)

    # Display the results
    print(df)

def filter_stocks(): 
    filtered_stocks = set()
    # for index, stock in SP.iterrows():
    #     today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
    #     if today_change is not None and rolling_avg is not None:  # Only process if valid data is returned
    #         if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
    #             filtered_stocks.add(stock['Symbol'])

    for index, stock in DOWJ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    for index, stock in NASDAQ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    return filtered_stocks

def get_price_change_and_rolling_avg(ticker: str):
    try:
        # Define the date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=24)
        
        # Download stock data
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        # If data is empty, return None
        if data.empty:
            return None, None
        
        data = data.sort_index()

        # Calculate the daily price change
        data['Price_Change'] = (data['Close'].diff() / data['Close'].shift(1)) * 100
        today_price_change = data['Price_Change'].iloc[-1]

        # Calculate the rolling average of the price change
        rolling_avg = data['Price_Change'].rolling(window=min(14, len(data))).mean().iloc[-1]

        return today_price_change, rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def fetch_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{underlying_symbol}?feed=indicative&limit=100&expiration_date=2025-02-07"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        snapshots = data.get('snapshots', {})
        return snapshots
    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None

def parse_option_chain(option_chain):

    parsed_data = []
    
    for symbol, details in option_chain.items():
        expiration_start = len(symbol) - 15
        ticker = symbol[:expiration_start]
        expiration_date = f"20{symbol[expiration_start:expiration_start+2]}-{symbol[expiration_start+2:expiration_start+4]}-{symbol[expiration_start+4:expiration_start+6]}"
        option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
        strike_price = int(symbol[expiration_start+7:]) / 1000  
        
        greeks = details.get("greeks", {}) or {}
        implied_volatility = details.get("impliedVolatility", None)
        latest_quote = details.get("latestQuote", {})
        
        parsed_data.append({
            "ticker": ticker,
            "expiration_date": expiration_date,
            "option_type": option_type,
            "strike_price": strike_price,
            "delta": greeks.get("delta"),
            "gamma": greeks.get("gamma"),
            "rho": greeks.get("rho"),
            "theta": greeks.get("theta"),
            "vega": greeks.get("vega"),
            "implied_volatility": implied_volatility,
            "ask_price": latest_quote.get("ap"),
            "ask_size": latest_quote.get("as"),
            "bid_price": latest_quote.get("bp"),
            "bid_size": latest_quote.get("bs")
        })
    
    return pd.DataFrame(parsed_data)

def get_option_chain(api_key: str, secret_key: str, underlying_symbol: str):
    option_chain = fetch_option_chain(api_key, secret_key, underlying_symbol)
    if option_chain:
        return parse_option_chain(option_chain)
    else:
        return None
    

def gbm(s0, mu, sigma, deltaT, dt):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return s


def objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)


def archive_objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)

def archive_optimize_gbm(symbol):
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 5 years)
    stock_data = yf.download(symbol, period="2y", interval="1d")
    real_prices = stock_data["Close"].dropna().values

    # Split into bins of 20 trading days
    bin_size = 20
    num_bins = len(real_prices) // bin_size
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    mu_values, sigma_values, mses = [], [], []

    # Optimize each bin
    for i in range(num_bins):
        bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
        s0 = bin_prices[0]

        # Define the bounds for optimization
        bounds = [(-0.3, 0.3), (0.001, 0.35)]

        # Run the optimizer
        result = differential_evolution(objective, bounds, args=(bin_prices, s0))
        best_mu, best_sigma = result.x
        best_mse = result.fun

        mu_values.append(best_mu)
        sigma_values.append(best_sigma)
        mses.append(best_mse)

        print(f"Bin {i+1}/{num_bins}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")

    # Compute weighted averages
    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma

def gbm_vs_real_graph(symbol, mu, sigma, period):
    stock_data = yf.download(symbol, period=period, interval="1d")
    real_prices = stock_data["Close"].dropna().values
    time_steps = np.arange(len(real_prices))


    gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
    plt.figure(figsize=(10, 5))
    plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
    plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
    plt.xlabel("Time (Days)")
    plt.ylabel("Price")
    plt.title(f"GBM vs Real Prices for {symbol}")
    plt.legend()
    plt.grid()
    plt.show()

# Example usage
# avg_mu, avg_sigma = optimize_gbm("AAPL")
#  gbm_vs_real_graph('AAPL', mu=0.0011, sigma=0.0130, period='2y')

In [61]:
candidates = filter_stocks()
simulation_attempts = 100
risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100
all_options = pd.DataFrame(columns=['ticker', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
pd.set_option('display.max_rows', None)
print(candidates)

for symbol in candidates:
    option_chain = get_option_chain(api_key, secret_key, symbol)
    put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    if option_chain is None or option_chain.empty:
        continue 

    price = get_current_stock_price(symbol)
    optimized_mu, optimized_sigma = archive_optimize_gbm(symbol)

    profitability_chances = []
    percent_returns = []

    for index, contract in put_chain.iterrows():
        count = 0
        strike_price = contract['strike_price']

        for i in range(simulation_attempts):
            prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)
        
            if prices[-1] > strike_price:
                count += 1

        profitability_chance = (count / simulation_attempts) * 100
        profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
        percent_return = (profit / (strike_price)) * 100

        profitability_chances.append(profitability_chance)
        percent_returns.append(percent_return)

    put_chain['profitability_percent'] = profitability_chances
    put_chain['percent_return'] = percent_returns

    if put_chain['percent_return'].std() != 0:
        put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    else:
        put_chain['sharpe_ratio'] = 0  # Avoid division by zero

    # filename = f"/Options/{symbol}_put_options.csv"
    # put_chain.to_csv(filename, index=False)

    # print(f"Saved {symbol} data to {filename}")

    all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)



{'INTC', 'PDD', 'AMD'}


[*********************100%***********************]  1 of 1 completed


Bin 1/25: μ = -0.0102, σ = 0.0239, MSE = 0.2482
Bin 2/25: μ = 0.0123, σ = 0.0184, MSE = 0.2382
Bin 3/25: μ = -0.0056, σ = 0.0087, MSE = 0.1384
Bin 4/25: μ = -0.0008, σ = 0.0153, MSE = 0.2197
Bin 5/25: μ = 0.0016, σ = 0.0214, MSE = 0.4785
Bin 6/25: μ = 0.0011, σ = 0.0080, MSE = 0.1731
Bin 7/25: μ = -0.0054, σ = 0.0066, MSE = 0.2550
Bin 8/25: μ = -0.0035, σ = 0.0236, MSE = 0.4627
Bin 9/25: μ = 0.0027, σ = 0.0121, MSE = 0.1012
Bin 10/25: μ = 0.0144, σ = 0.0173, MSE = 0.5094
Bin 11/25: μ = -0.0008, σ = 0.0159, MSE = 0.3682
Bin 12/25: μ = 0.0042, σ = 0.0188, MSE = 0.6692
Bin 13/25: μ = -0.0041, σ = 0.0194, MSE = 0.7177
Bin 14/25: μ = -0.0002, σ = 0.0148, MSE = 0.4492
Bin 15/25: μ = -0.0054, σ = 0.0186, MSE = 0.4068
Bin 16/25: μ = -0.0120, σ = 0.0145, MSE = 0.2623
Bin 17/25: μ = -0.0022, σ = 0.0131, MSE = 0.1277
Bin 18/25: μ = 0.0057, σ = 0.0116, MSE = 0.2760
Bin 19/25: μ = -0.0256, σ = 0.0464, MSE = 2.0813
Bin 20/25: μ = 0.0001, σ = 0.0202, MSE = 0.2672
Bin 21/25: μ = 0.0141, σ = 0.0189, MS

/var/folders/90/x6spqd2j4cbb38yrckqjm8840000gn/T/ipykernel_17550/2874708714.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)
[*********************100%***********************]  1 of 1 completed


Bin 1/25: μ = -0.0020, σ = 0.0250, MSE = 3.7941
Bin 2/25: μ = -0.0173, σ = 0.0209, MSE = 8.2061
Bin 3/25: μ = -0.0092, σ = 0.0167, MSE = 1.1759
Bin 4/25: μ = 0.0025, σ = 0.0315, MSE = 3.8236
Bin 5/25: μ = 0.0030, σ = 0.0389, MSE = 3.3049
Bin 6/25: μ = 0.0122, σ = 0.0171, MSE = 2.7365
Bin 7/25: μ = -0.0081, σ = 0.0074, MSE = 1.2471
Bin 8/25: μ = 0.0166, σ = 0.0430, MSE = 11.4502
Bin 9/25: μ = 0.0011, σ = 0.0188, MSE = 3.4695
Bin 10/25: μ = 0.0072, σ = 0.0124, MSE = 2.5373
Bin 11/25: μ = 0.0142, σ = 0.0284, MSE = 13.6019
Bin 12/25: μ = -0.0032, σ = 0.0101, MSE = 2.4883
Bin 13/25: μ = -0.0063, σ = 0.0245, MSE = 6.4988
Bin 14/25: μ = -0.0059, σ = 0.0224, MSE = 5.1454
Bin 15/25: μ = -0.0047, σ = 0.0187, MSE = 3.4288
Bin 16/25: μ = 0.0088, σ = 0.0134, MSE = 8.6074
Bin 17/25: μ = 0.0012, σ = 0.0127, MSE = 4.9469
Bin 18/25: μ = -0.0063, σ = 0.0109, MSE = 3.2913
Bin 19/25: μ = -0.0008, σ = 0.0176, MSE = 4.9567
Bin 20/25: μ = -0.0226, σ = 0.0617, MSE = 50.0963
Bin 21/25: μ = 0.0219, σ = 0.0386, 

[*********************100%***********************]  1 of 1 completed


Bin 1/25: μ = -0.0041, σ = 0.0068, MSE = 2.3754
Bin 2/25: μ = 0.0097, σ = 0.0182, MSE = 3.1979
Bin 3/25: μ = -0.0061, σ = 0.0096, MSE = 1.4640
Bin 4/25: μ = 0.0212, σ = 0.0238, MSE = 7.3810
Bin 5/25: μ = -0.0040, σ = 0.0213, MSE = 6.4882
Bin 6/25: μ = -0.0013, σ = 0.0148, MSE = 2.1223
Bin 7/25: μ = -0.0038, σ = 0.0096, MSE = 4.4147
Bin 8/25: μ = -0.0013, σ = 0.0190, MSE = 2.6351
Bin 9/25: μ = 0.0055, σ = 0.0197, MSE = 2.1956
Bin 10/25: μ = 0.0088, σ = 0.0209, MSE = 6.5265
Bin 11/25: μ = 0.0083, σ = 0.0198, MSE = 4.0411
Bin 12/25: μ = 0.0092, σ = 0.0184, MSE = 11.7786
Bin 13/25: μ = 0.0007, σ = 0.0109, MSE = 7.9638
Bin 14/25: μ = 0.0115, σ = 0.0318, MSE = 29.2637
Bin 15/25: μ = -0.0058, σ = 0.0145, MSE = 5.3390
Bin 16/25: μ = -0.0062, σ = 0.0198, MSE = 11.8381
Bin 17/25: μ = 0.0050, σ = 0.0132, MSE = 7.3522
Bin 18/25: μ = 0.0039, σ = 0.0166, MSE = 8.1185
Bin 19/25: μ = -0.0172, σ = 0.0152, MSE = 13.7239
Bin 20/25: μ = -0.0051, σ = 0.0196, MSE = 9.1670
Bin 21/25: μ = 0.0093, σ = 0.0132, 

In [64]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""

all_options


,ticker,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_percent,percent_return,sharpe_ratio
0,INTC,2025-02-07,Put,17.0,-0.0499,0.0632,-0.0001,-0.0208,0.0021,0.7999,0.04,202,0.03,478,100.0,0.193945,0.035084
1,INTC,2025-02-07,Put,17.5,-0.0592,0.0879,-0.0001,-0.0196,0.0024,0.6589,0.06,1322,0.01,1564,100.0,0.188021,0.033861
2,INTC,2025-02-07,Put,18.0,-0.1320,0.1541,-0.0003,-0.0368,0.0043,0.6818,0.10,1981,0.09,1725,99.0,0.529697,0.104381
3,INTC,2025-02-07,Put,18.5,-0.2222,0.2249,-0.0005,-0.0487,0.0061,0.6506,0.18,2048,0.17,2040,93.0,0.945999,0.190304
4,INTC,2025-02-07,Put,19.0,-0.3484,0.2942,-0.0008,-0.0573,0.0075,0.6179,0.33,332,0.28,448,73.0,1.585695,0.322333
5,INTC,2025-02-07,Put,19.5,-0.5039,0.3084,-0.0011,-0.0633,0.0081,0.6357,0.55,368,0.54,833,42.0,2.784944,0.569852
6,INTC,2025-02-07,Put,20.0,-0.6509,0.2845,-0.0015,-0.0586,0.0075,0.6393,0.86,735,0.84,465,21.0,4.261250,0.874554
7,INTC,2025-02-07,Put,20.5,-0.7714,0.2289,-0.0018,-0.0481,0.0062,0.6497,1.25,262,1.20,866,6.0,5.910310,1.214911
8,INTC,2025-02-07,Put,21.0,-0.8501,0.1671,-0.0020,-0.0385,0.0047,0.6855,1.68,436,1.64,432,0.0,7.905201,1.626646
9,INTC,2025-02-07,Put,21.5,-0.8851,0.1233,-0.0021,-0.0359,0.0039,0.7733,2.16,123,2.12,108,0.0,9.959529,2.050648


In [63]:
# Archive

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 2 years)
    stock_data = yf.download(symbol, period="2y", interval="1d")
    real_prices = stock_data["Close"].dropna().values

    # Split into bins of 20 trading days
    bin_size = 20
    num_bins = len(real_prices) // bin_size
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # Initialize containers for results
    mu_values, sigma_values, mses = [], [], []

    # Use concurrent.futures for parallel processing of bins
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(num_bins):
            bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
            futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
        for future in concurrent.futures.as_completed(futures):
            best_mu, best_sigma, best_mse = future.result()
            mu_values.append(best_mu)
            sigma_values.append(best_sigma)
            mses.append(best_mse)

    # Compute weighted averages
    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma